# Linear Regression in Scala

### Author: Kovalev Evgeny

In this homework, I needed to analyze and process the data, fit linear regression and estimate quality of the model. I decided to use RMSLE (Root Mean Squared Logarithmic Error) as a quality metric.

I used almond Scala 2.12 kernel, Apache Spark for data processing and Breeze for constructing linear regression.

_I didn't manage to create plots which I needed in Scala, so I used Python for them instead._

## 1. Data analysis

### 1.1. Setting up

Set up Spark and import `breeze`:

In [1]:
// https://github.com/Gorini4/jupyter_scala_docker

import $ivy.`org.apache.spark::spark-mllib:3.1.2`
import $ivy.`org.apache.spark::spark-sql:2.4.6`
import $ivy.`org.scalanlp::breeze:1.2`
import $ivy.`org.scalanlp::breeze-natives:1.2`
import $ivy.`org.scalanlp::breeze-viz:1.2`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.sql.functions.{col, desc, log1p}

import breeze.linalg.{DenseMatrix, DenseVector, pinv}
import breeze.stats.distributions.Gaussian

import scala.math.{log, sqrt}

val spark = {
  AmmoniteSparkSession.builder()
    .master("local[*]")
    .config("spark.ui.showConsoleProgress", "false")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.30/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.16/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                    

import $ivy.$                                  

import $ivy.$                         

import $ivy.$                                 

import $ivy.$                             


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.sql.functions.{col, desc, log1p}


import breeze.linalg.{DenseMatrix, DenseVector, pinv}

import breeze.stats.distributions.Gaussian


import scala.math.{log, sqrt}


spark: SparkSession = org.apache.spark.sql.SparkSession@77f2d6d5

A function for visualizing dataframe:

In [2]:
// https://github.com/almond-sh/almond/issues/180

implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit:Int = 20, truncate: Int = 20) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map { cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }

        publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map { row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)        
    }
}

defined class RichDF

### 1.2. Basic data things

Read dataframe from csv:

In [3]:
// https://stackoverflow.com/questions/29704333/spark-load-csv-file-as-dataframe

val df = spark.read
         .format("csv")
         .format("com.databricks.spark.csv")
         .option("inferSchema","true") 
         .option("header", "true") //first line in file has headers
         .load("data/Dataset.csv")

df.showHTML()

likes,Checkins,Returns,Category,commBase,comm24,comm48,comm24_1,diff2448,baseTime,length,shares,hrs,sun_pub,mon_pub,tue_pub,wed_pub,thu_pub,fri_pub,sat_pub,sun_base,mon_base,tue_base,wed_base,thu_base,fri_base,sat_base,output
634995,0,463,1,0,0,0,0,0,65,166,2,24,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
634995,0,463,1,0,0,0,0,0,10,132,1,24,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
634995,0,463,1,0,0,0,0,0,14,133,2,24,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
634995,0,463,1,7,0,3,7,-3,62,131,1,24,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
634995,0,463,1,1,0,0,1,0,58,142,5,24,0,null,0,0,0,0,0,0,0,0,1,0,0,0,0
634995,0,463,1,0,0,null,0,0,60,166,1,24,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
634995,0,463,1,0,0,null,0,0,68,145,2,24,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
634995,0,463,1,1,0,1,1,-1,32,157,2,24,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
634995,0,463,1,0,0,null,0,0,35,177,5,24,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
634995,0,463,1,0,0,null,0,0,48,126,1,24,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0


df: DataFrame = [likes: int, Checkins: int ... 26 more fields]

Print dataframe shape (number of rows and number of columns):

In [4]:
println(df.count(), df.columns.size)

(40949,28)


Print all columns names:

In [5]:
df.columns.toSeq

res4: Seq[String] = WrappedArray(
  "likes",
  "Checkins",
  "Returns",
  "Category",
  "commBase",
  "comm24",
  "comm48",
  "comm24_1",
  "diff2448",
  "baseTime",
  "length",
  "shares",
  "hrs",
  "sun_pub",
  "mon_pub",
  "tue_pub",
  "wed_pub",
  "thu_pub",
  "fri_pub",
  "sat_pub",
  "sun_base",
  "mon_base",
  "tue_base",
  "wed_base",
  "thu_base",
  "fri_base",
  "sat_base",
  "output"
)

Get columns datatypes:

In [6]:
df.dtypes

res5: Array[(String, String)] = Array(
  ("likes", "IntegerType"),
  ("Checkins", "IntegerType"),
  ("Returns", "IntegerType"),
  ("Category", "IntegerType"),
  ("commBase", "IntegerType"),
  ("comm24", "IntegerType"),
  ("comm48", "IntegerType"),
  ("comm24_1", "IntegerType"),
  ("diff2448", "IntegerType"),
  ("baseTime", "IntegerType"),
  ("length", "IntegerType"),
  ("shares", "IntegerType"),
  ("hrs", "IntegerType"),
  ("sun_pub", "IntegerType"),
  ("mon_pub", "IntegerType"),
  ("tue_pub", "IntegerType"),
  ("wed_pub", "IntegerType"),
  ("thu_pub", "IntegerType"),
  ("fri_pub", "IntegerType"),
  ("sat_pub", "IntegerType"),
  ("sun_base", "IntegerType"),
  ("mon_base", "IntegerType"),
  ("tue_base", "IntegerType"),
  ("wed_base", "IntegerType"),
  ("thu_base", "IntegerType"),
  ("fri_base", "IntegerType"),
  ("sat_base", "IntegerType"),
  ("output", "IntegerType")
)

Summary of the target column:

In [7]:
df.select("output").summary().show()

+-------+-----------------+
|summary|           output|
+-------+-----------------+
|  count|            40949|
|   mean|7.322889447849764|
| stddev|35.49454978260473|
|    min|                0|
|    25%|                0|
|    50%|                0|
|    75%|                3|
|    max|             1305|
+-------+-----------------+



Correlations of features in the dataframe:

<img src="images/correlations.jpg">

### 1.3. Checking NaNs

Function for checking if there are any missing values in some column:

In [8]:
val get_nans = (df: DataFrame, col_name: String) => df.withColumn(
    col_name + "_null", col(col_name).isNull
).groupBy(col_name + "_null").count()

get_nans: (DataFrame, String) => DataFrame = ammonite.$sess.cmd7$Helper$$Lambda$5709/1449470737@435ecb87

Are there any missing values in the target column?

In [9]:
get_nans(df, "output").show()

+-----------+-----+
|output_null|count|
+-----------+-----+
|      false|40949|
+-----------+-----+



Ok, target column has no NaNs. Let us write a method in order to print number of NaNs in every dataframe column in a pretty way:

In [10]:
def getNaNs(df: DataFrame, col_name: String): String = {
    val df_col = get_nans(df, col_name)
    if (df_col.count > 1) {
        val nan_count = df_col.where(col_name + "_null == true").first()(1)
        s"$col_name: $nan_count NaNs"
    } else {
        s"$col_name: no NaNs"
    }
}
for (col_name <- df.columns.toSeq) println(getNaNs(df, col_name))

likes: no NaNs
Checkins: no NaNs
Returns: 51 NaNs
Category: 57 NaNs
commBase: 60 NaNs
comm24: no NaNs
comm48: 48 NaNs
comm24_1: no NaNs
diff2448: no NaNs
baseTime: no NaNs
length: no NaNs
shares: 2449 NaNs
hrs: no NaNs
sun_pub: no NaNs
mon_pub: 1927 NaNs
tue_pub: no NaNs
wed_pub: no NaNs
thu_pub: 3045 NaNs
fri_pub: no NaNs
sat_pub: no NaNs
sun_base: no NaNs
mon_base: 1970 NaNs
tue_base: no NaNs
wed_base: no NaNs
thu_base: no NaNs
fri_base: no NaNs
sat_base: no NaNs
output: no NaNs


defined function getNaNs

### 1.4. Target variable

Most popular values of the target variable (with counts sorted in the descending order):

In [11]:
// https://stackoverflow.com/questions/51725418/dataframe-how-to-groupby-count-then-order-by-count-in-scala/51726490
df.groupBy("output").count().orderBy(desc("count")).show()

+------+-----+
|output|count|
+------+-----+
|     0|22579|
|     1| 5194|
|     2| 2624|
|     3| 1583|
|     4| 1171|
|     5|  836|
|     6|  636|
|     7|  564|
|     8|  414|
|     9|  374|
|    10|  307|
|    11|  300|
|    13|  235|
|    12|  234|
|    15|  220|
|    14|  184|
|    16|  147|
|    17|  143|
|    20|  126|
|    21|  123|
+------+-----+
only showing top 20 rows



Quantiles for the target variable:

In [12]:
// https://towardsdatascience.com/exploratory-data-analysis-eda-with-pyspark-on-databricks-e8d6529626b1
val quantiles = df.stat.approxQuantile("output", Array(0.25, 0.5, 0.75), 0)
println("25% quantile: " + quantiles(0))
println("50% quantile: " + quantiles(1))
println("75% quantile: " + quantiles(2))

25% quantile: 0.0
50% quantile: 0.0
75% quantile: 3.0


quantiles: Array[Double] = Array(0.0, 0.0, 3.0)

We can see that the distribution of the target variable is extremely skewed towards zero. Let's check it by plotting a histogram of logarithm of its values:

<img src="images/histogram_log-output.jpg">

## 2. Data preprocessing

### 2.1. Filling NaNs

Fill missing values:

In [13]:
println("Before preprocessing | " + getNaNs(df, "Returns"))
println("After preprocessing | " + getNaNs(df.na.fill(0, Seq("Returns")), "Returns"))

Before preprocessing | Returns: 51 NaNs
After preprocessing | Returns: no NaNs


In [14]:
val df_nonans = df.na.fill(0)
for (col_name <- df_nonans.columns.toSeq) println(getNaNs(df_nonans, col_name))

likes: no NaNs
Checkins: no NaNs
Returns: no NaNs
Category: no NaNs
commBase: no NaNs
comm24: no NaNs
comm48: no NaNs
comm24_1: no NaNs
diff2448: no NaNs
baseTime: no NaNs
length: no NaNs
shares: no NaNs
hrs: no NaNs
sun_pub: no NaNs
mon_pub: no NaNs
tue_pub: no NaNs
wed_pub: no NaNs
thu_pub: no NaNs
fri_pub: no NaNs
sat_pub: no NaNs
sun_base: no NaNs
mon_base: no NaNs
tue_base: no NaNs
wed_base: no NaNs
thu_base: no NaNs
fri_base: no NaNs
sat_base: no NaNs
output: no NaNs


df_nonans: DataFrame = [likes: int, Checkins: int ... 26 more fields]

### 2.2. Transforming target

During data analysis, we saw that target variable distribution is extremely skewed. Let's perform log-transformation of the target variable:

In [15]:
df_nonans.withColumn("output_log", log1p("output"))

res14: DataFrame = [likes: int, Checkins: int ... 27 more fields]

In [16]:
// log1p is used (log(x + 1)) to avoid NaNs if x = 0

val df_log = df_nonans.withColumn("output_log", log1p("output")).na.fill(-1)
df_log.select("output", "output_log").showHTML(30)

output,output_log
0,0.0
0,0.0
0,0.0
0,0.0
0,0.0
0,0.0
0,0.0
0,0.0
0,0.0
0,0.0


df_log: DataFrame = [likes: int, Checkins: int ... 27 more fields]

Drop the original target variable:

In [17]:
val df_logtarget = df_log.drop("output")
df_logtarget.showHTML()

likes,Checkins,Returns,Category,commBase,comm24,comm48,comm24_1,diff2448,baseTime,length,shares,hrs,sun_pub,mon_pub,tue_pub,wed_pub,thu_pub,fri_pub,sat_pub,sun_base,mon_base,tue_base,wed_base,thu_base,fri_base,sat_base,output_log
634995,0,463,1,0,0,0,0,0,65,166,2,24,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.0
634995,0,463,1,0,0,0,0,0,10,132,1,24,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0.0
634995,0,463,1,0,0,0,0,0,14,133,2,24,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.0
634995,0,463,1,7,0,3,7,-3,62,131,1,24,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0.0
634995,0,463,1,1,0,0,1,0,58,142,5,24,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0
634995,0,463,1,0,0,0,0,0,60,166,1,24,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.0
634995,0,463,1,0,0,0,0,0,68,145,2,24,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.0
634995,0,463,1,1,0,1,1,-1,32,157,2,24,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0.0
634995,0,463,1,0,0,0,0,0,35,177,5,24,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.0
634995,0,463,1,0,0,0,0,0,48,126,1,24,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0.0


df_logtarget: DataFrame = [likes: int, Checkins: int ... 26 more fields]

### 2.3. Changing columns type

Convert all columns to double in order to form `DenseMatrix` in `breeze` further:

In [18]:
val df_processed = df_logtarget.select(df_logtarget.columns.toSeq.map(
    c => col(c).cast("double")
): _*)
df_processed.showHTML()

likes,Checkins,Returns,Category,commBase,comm24,comm48,comm24_1,diff2448,baseTime,length,shares,hrs,sun_pub,mon_pub,tue_pub,wed_pub,thu_pub,fri_pub,sat_pub,sun_base,mon_base,tue_base,wed_base,thu_base,fri_base,sat_base,output_log
634995.0,0.0,463.0,1.0,0.0,0.0,0.0,0.0,0.0,65.0,166.0,2.0,24.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
634995.0,0.0,463.0,1.0,0.0,0.0,0.0,0.0,0.0,10.0,132.0,1.0,24.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
634995.0,0.0,463.0,1.0,0.0,0.0,0.0,0.0,0.0,14.0,133.0,2.0,24.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
634995.0,0.0,463.0,1.0,7.0,0.0,3.0,7.0,-3.0,62.0,131.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
634995.0,0.0,463.0,1.0,1.0,0.0,0.0,1.0,0.0,58.0,142.0,5.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
634995.0,0.0,463.0,1.0,0.0,0.0,0.0,0.0,0.0,60.0,166.0,1.0,24.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
634995.0,0.0,463.0,1.0,0.0,0.0,0.0,0.0,0.0,68.0,145.0,2.0,24.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
634995.0,0.0,463.0,1.0,1.0,0.0,1.0,1.0,-1.0,32.0,157.0,2.0,24.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
634995.0,0.0,463.0,1.0,0.0,0.0,0.0,0.0,0.0,35.0,177.0,5.0,24.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
634995.0,0.0,463.0,1.0,0.0,0.0,0.0,0.0,0.0,48.0,126.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


df_processed: DataFrame = [likes: double, Checkins: double ... 26 more fields]

## 3. Linear regression

### 3.1. Getting X and y

First, let's construct dense feature matrix `X`. I could not come up with the idea how to perform a similar operation for many columns, so I decided to concatenate two supplementary dense matrices and bias vector.

In [19]:
val dm1 = DenseMatrix(df_processed.select(
    "likes", "Checkins", "Returns", "Category", "commBase",
    "comm24", "comm48", "comm24_1", "diff2448", "baseTime",
    "length", "shares", "hrs", "sun_pub"
).collect().map(r => (
    r.getDouble(0), r.getDouble(1), r.getDouble(2), r.getDouble(3), r.getDouble(4),
    r.getDouble(5), r.getDouble(6), r.getDouble(7), r.getDouble(8), r.getDouble(9),
    r.getDouble(10), r.getDouble(11), r.getDouble(12), r.getDouble(13)
)): _*)

val dm2 = DenseMatrix(df_processed.select(
    "mon_pub", "tue_pub", "wed_pub", "thu_pub", "fri_pub",
    "sat_pub", "sun_base", "mon_base", "tue_base", "wed_base",
    "thu_base", "fri_base", "sat_base"
).collect().map(r => (
    r.getDouble(0), r.getDouble(1), r.getDouble(2), r.getDouble(3), r.getDouble(4),
    r.getDouble(5), r.getDouble(6), r.getDouble(7), r.getDouble(8), r.getDouble(9),
    r.getDouble(10), r.getDouble(11), r.getDouble(12)
)): _*)

val bias = DenseMatrix.ones[Double](dm1.rows, 1)

dm1: DenseMatrix[Double] = 634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    65.0  ... (14 total)
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    10.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    14.0  ...
634995.0  0.0  463.0  1.0  7.0   0.0   3.0   7.0   -3.0   62.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    58.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    60.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    68.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   32.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    35.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    48.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    52.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    69.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    3.0   ...
634995.0  0.0  463.0  1.0  1.0   1.0   0.0   1.0   1.0    37.0  ...
634995.0  

In [20]:
val X = DenseMatrix.horzcat(dm1, dm2, bias)
println(X.rows, X.cols)

(40949,28)


X: DenseMatrix[Double] = 634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    65.0  ... (28 total)
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    10.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    14.0  ...
634995.0  0.0  463.0  1.0  7.0   0.0   3.0   7.0   -3.0   62.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    58.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    60.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    68.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   32.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    35.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    48.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    52.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    69.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    3.0   ...
634995.0  0.0  463.0  1.0  1.0   1.0   0.0   1.0   1.0    37.0  ...
634995.0  0.

Get target vector:

In [21]:
val target = DenseVector(df_processed.select("output_log").collect().map(r => r.getDouble(0)): _*)

target: DenseVector[Double] = DenseVector(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3862943611198906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.791759469228055, 0.0, 0.0, 0.0, 0.0, 1.0986122886681096, 0.0, 0.0, 0.0, 0.0, 2.772588722239781, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0986122886681096, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0986122886681096, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0986122886681096, 0.6931471805599453, 0.0, 0.0, 1.3862943611198906, 0.0, 0.6931471805599453, 0.0, 0.0, 1.3862943611198906, 0.6931471805599453, 0.6931471805599453, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0986122886681096, 1.0986122886681096, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6931471805599453, 1.0986122886681096, 0.0, 0.0, 0.0, 0.6931471805599453, 0.0, 0.0, 0.6931471805599453, 0.0, 1.3862943611198906, 0.0, 0.0, 0.0, 0.6931471805599453, 0.0, 0.6931471805599453, 2.302585092994046, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0986122886681096, 0.0, 0.0, 0.0, 1

### 3.2. Scaling X

We can notice that features in `X` matrix have different scale:

In [22]:
for (i <- 0 to (X.cols - 2)) {
    val col_mean_i = X(::, i).reduce(_ + _) / X(::, i).length
    val col_std_i = (
        X(::, i) - DenseVector(Array.fill[Double](X.rows)(col_mean_i))
    ).map(x => x * x).reduce(_ + _) / X.rows
    println(s"Mean, std of column $i: $col_mean_i, $col_std_i")
}

Mean, std of column 0: 1313813.7475396225, 4.604530230108423E13
Mean, std of column 1: 4676.133751739969, 4.240689065247651E8
Mean, std of column 2: 44800.24376663655, 1.2306007625815887E10
Mean, std of column 3: 24.238662726806517, 398.2900815447012
Mean, std of column 4: 55.712032039854456, 18762.75738801233
Mean, std of column 5: 22.18664680456177, 5948.006653603457
Mean, std of column 6: 19.649539671298445, 5052.045899628512
Mean, std of column 7: 52.631590515030894, 16429.690594993812
Mean, std of column 8: 2.5369606095386943, 8873.983523656447
Mean, std of column 9: 35.32203472612274, 437.50451361612926
Mean, std of column 10: 163.6524701457911, 141571.43127556986
Mean, std of column 11: 117.12146816772082, 893045.5529025972
Mean, std of column 12: 23.767833158318883, 3.685652637086797
Mean, std of column 13: 0.12234731006862194, 0.10737844578756893
Mean, std of column 14: 0.12036923978607536, 0.10588048589944102
Mean, std of column 15: 0.14974724657500793, 0.1273230087182284
Mea

Thus let's perform standard scaling - subtract mean and divide by std each column $x_i$ of `X` (except bias):

$$
x_i = \frac{x_i - \text{mean}(x_i)}{\text{std}(x_i)}
$$

In [23]:
for (i <- 0 to (X.cols - 2)) {
    val col_mean_i = X(::, i).reduce(_ + _) / X(::, i).length
    val col_std_i = (
        X(::, i) - DenseVector(Array.fill[Double](X.rows)(col_mean_i))
    ).map(x => x * x).reduce(_ + _) / X.rows
    X(::, i) := (
        X(::, i) - DenseVector(Array.fill[Double](X.rows)(col_mean_i))
    ) / DenseVector(Array.fill[Double](X.rows)(col_std_i))
}

Check the results:

In [24]:
for (i <- 0 to (X.cols - 2)) {
    val col_mean_i = X(::, i).reduce(_ + _) / X(::, i).length
    val col_std_i = (
        X(::, i) - DenseVector(Array.fill[Double](X.rows)(col_mean_i))
    ).map(x => x * x).reduce(_ + _) / X.rows
    println(s"Mean, std of column $i: $col_mean_i, $col_std_i")
}

Mean, std of column 0: -6.737644551622921E-23, 2.171774209367018E-14
Mean, std of column 1: 6.004380318896852E-19, 2.358107337306568E-9
Mean, std of column 2: 3.87059037071051E-21, 8.126112305522776E-11
Mean, std of column 3: 5.969366955093793E-16, 0.0025107328711815495
Mean, std of column 4: 2.1138132338982717E-17, 5.329707032501064E-5
Mean, std of column 5: 3.5991625319052855E-17, 1.6812355100412048E-4
Mean, std of column 6: 3.1849538269622874E-17, 1.9793961097477674E-4
Mean, std of column 7: 2.2929579904298288E-17, 6.086541887192017E-5
Mean, std of column 8: -3.324650135404035E-19, 1.1268896289179112E-4
Mean, std of column 9: 2.9336394224863585E-19, 0.0022856907046160052
Mean, std of column 10: -1.8131129425577618E-18, 7.063572014422638E-6
Mean, std of column 11: 3.50702295437593E-19, 1.1197637083014907E-6
Mean, std of column 12: -2.9480869082429613E-15, 0.27132236769623935
Mean, std of column 13: -1.8284548187208375E-14, 9.312855970913379
Mean, std of column 14: 9.822289424133222E-

This is much better - we can see that all means are around zero and standard deviations are not so big.

### 3.3. Quality metric

As a quality metric, I decided to use a pretty standard RMSE. However, since we performed log transformation of target, it becomes RMSLE (RMSE on log-targets).

Let's generate a random vector with weights and calculate RMSLE:

In [25]:
val w = DenseVector(Gaussian(0.0,1.0).sample(X.cols).toArray)

// https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/8623654525287098/4373605817327958/8746817301327119/latest.html

def calculateRMSLE(X: DenseMatrix[Double], y: DenseVector[Double], w: DenseVector[Double]): Double = {
    val residuals = X * w - target
    val rmsle = sqrt(residuals.map(x => x * x).reduce(_ + _) / residuals.length)
    rmsle
}

println(calculateRMSLE(X, target, w))

21/06/14 15:18:39 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/06/14 15:18:39 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


8.461787900655793


w: DenseVector[Double] = DenseVector(-0.22801028404242707, 0.8917826998425835, 0.6590668623433276, 1.4887770774767686, 1.0970490378117443, -1.3634934607706388, 0.8325157949187117, -0.5738118191617432, 0.7860910693655725, 0.10191685248508559, -1.3993552113271648, 0.32129351706257975, -0.38793135696241693, 0.638093129694175, -1.7688859642469306, -1.743910264517766, -0.14194577044234596, -0.9694645795349695, -0.7048142992830599, -0.471880663056795, -1.6687859863855858, -0.3677601607482501, 0.22041821930243535, 0.5791813017644524, 0.7324636138882982, 0.21589344952591064, 0.4506003183601724, 1.7527420334335948)
defined function calculateRMSLE

### 3.4. Linear Regression

Finally, let's estimate weights according to the following formula:

$$
\hat{w} = \left(X^TX\right)^{-1}X^Ty
$$

In [26]:
val w_hat = pinv(X.t * X) * X.t * target

21/06/14 15:18:40 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/06/14 15:18:40 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


w_hat: DenseVector[Double] = DenseVector(-153876.98519518785, 247.4862230863165, 15285.259134671296, -1.600776387967798, 193.13741415074847, 534.9723800728724, -421.04765447710975, -155.40304862121866, -745.7131599658776, -10.25922272678158, 6.88625786502682, -15.07578567454206, 0.09627311699629464, -0.007115344298176117, -0.008089431208170118, -0.008959306180004411, -0.007447259500170732, -0.003846888718764723, -0.005689792545439303, -0.006828018002001336, -0.006673614492336606, -0.004701287014236966, -0.00907858775797032, -0.0076403881391018036, -0.006266660481077794, -0.00802630927519898, -0.007815401072763942, 0.8154149008981803)

Calculate RMSLE for them:

In [27]:
println(calculateRMSLE(X, target, w_hat))

0.8294968930349942


We can see that we got much better result than in the random weights case.

### 3.5. Ridge Regression

We can also add a regularization constant and construct Ridge regression. Weights estimation formula in this case looks like this:

$$
\hat{w} = \left(X^TX + \lambda I\right)^{-1}X^Ty
$$

Let's tune $\lambda$ and try to find an optimal one:

In [35]:
for (x <- 1 to 20) {
    val lambda = 1.0 / (x * x)
    val w_hat_ridge = pinv(X.t * X + DenseMatrix.eye[Double](X.cols) *:* lambda) * X.t * target
    val rmsle = calculateRMSLE(X, target, w_hat_ridge)
    println(s"Lambda: $lambda, RMSLE: $rmsle")
}

Lambda: 1.0, RMSLE: 0.8438610177336333
Lambda: 0.25, RMSLE: 0.8419898331546325
Lambda: 0.1111111111111111, RMSLE: 0.8414774704709597
Lambda: 0.0625, RMSLE: 0.8410408373724688
Lambda: 0.04, RMSLE: 0.8406114066536552
Lambda: 0.027777777777777776, RMSLE: 0.8401890030379445
Lambda: 0.02040816326530612, RMSLE: 0.8397817052638199
Lambda: 0.015625, RMSLE: 0.8393977696650294
Lambda: 0.012345679012345678, RMSLE: 0.8390434673449316
Lambda: 0.01, RMSLE: 0.8387224744406767
Lambda: 0.008264462809917356, RMSLE: 0.8384360328443419
Lambda: 0.006944444444444444, RMSLE: 0.8381834679242194
Lambda: 0.005917159763313609, RMSLE: 0.8379627921746231
Lambda: 0.00510204081632653, RMSLE: 0.8377712474217657
Lambda: 0.0044444444444444444, RMSLE: 0.8376057272788819
Lambda: 0.00390625, RMSLE: 0.8374630736980432
Lambda: 0.0034602076124567475, RMSLE: 0.8373402657522931
Lambda: 0.0030864197530864196, RMSLE: 0.8372345263336365
Lambda: 0.002770083102493075, RMSLE: 0.8371433713432606
Lambda: 0.0025, RMSLE: 0.8370646212017

Optimal lambda is 0.0025, but RMSLE is better for a regular linear regression.